In [1]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

computer =  blogin1
working directory set to /home/hbkoziel/awi-models


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [3]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True
# export CSV
export_csv = True
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/CSV/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [4]:
def dens0(T,S):
    T68 = T * 1.00024
    a0,a1,a2,a3,a4,a5 = 999.842594, 6.793952e-2, -9.095290e-3, 1.001685e-4, -1.120083e-6, 6.536332e-9
    sw_smow = a0 + (a1 + (a2 + (a3 + (a4 + a5*T68)*T68)*T68)*T68)*T68
    #     UNESCO 1983 eqn(13) p17.
    
    b0, b1, b2, b3, b4 =  8.24493e-1, -4.0899e-3, 7.6438e-5, -8.2467e-7, 5.3875e-9
    c0, c1, c2 = -5.72466e-3, 1.0227e-4, -1.6546e-6
    d0 = 4.8314e-4
    dens = sw_smow + (b0 + (b1 + (b2 + (b3 + b4*T68)*T68)*T68)*T68)*S + (c0 + (c1 + c2*T68)*T68)*S*np.sqrt(S) + d0*S**2
    return dens

In [5]:
mesh = fesom_mesh(meshpath, get3d = get3d)
#mesh = load_mesh(meshpath, usepickle=True,get3d=True)
mesh.n32 = mesh.n32-1
mesh.zlevs = -mesh.zlevs # depth is made negative

Load 2d part of the grid in 3 second(s)
Load 3d part of the grid in 41 seconds


In [ ]:
STRAT  = np.zeros((len(years),len(mesh.x2)))
for mo in range(3,9):
    print mo
    for ye in range(0,len(years)):
     print years[ye]
     ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
     f       = Dataset(ncfile, 'r')  
        
     if years[ye] < 2000:
        temp     = f.variables['temp'][mo,:]
        salt     = f.variables['salt'][mo,:]
        
     else:
        
        if mo ==3: 
            dayind = (range(45,60))
            month='APR'
        elif mo ==4: 
            dayind = (range(60,75))
            month='MAY'
        elif mo ==5: 
            dayind = (range(75,90))
            month='JUN'
        elif mo ==6: 
            dayind = (range(90,105))
            month='JUL'
        elif mo ==7:
            dayind = (range(105,120))
            month='AUG'
        elif mo ==8: 
            dayind = (range(120,135))
            month='SEP'
            
        temp     = f.variables['temp'][dayind,:].mean(axis=0)
        salt     = f.variables['salt'][dayind,:].mean(axis=0)
        
     dens = dens0(temp,salt)
     del salt
     del temp
     level_data1, elem_no_nan1 = pf.get_data(dens,mesh,0)
     level_data2, elem_no_nan2 = pf.get_data(dens,mesh,-100)
     strat = (level_data2-level_data1)
     STRAT[ye,:] = np.nan_to_num(strat)
     del level_data1
     del level_data2
     del elem_no_nan1
     del elem_no_nan2

    data2=STRAT.mean(axis=0)
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
    print 'Max and min: ',np.max(data2),np.min(data2)
    
    
    result_fit = np.ma.polyfit(years,STRAT,1)
    trend = result_fit[0,:]
    
    if export_csv == True:
        np.savetxt(outputpath+'STRAT_'+month+'_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv',STRAT, delimiter=";")
        np.savetxt(outputpath+'STRAT_'+month+'_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
        np.savetxt(outputpath+'STRAT_'+month+'_'+str(first_year)+'_'+str(last_year)+'_trend.csv', trend, delimiter=";")
        print 'exporting done ...'

3
1980
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1981
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1982
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1983
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1984
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1985
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1986
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1987
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1988
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1989
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1990
For depth 0 model level 0.0 will be used
For depth -100 model l

For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1994
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1995
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1996
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1997
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1998
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
1999
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2000
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2001
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2002
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2003
For depth 0 model level 0.0 will be used
For depth -100 model level -1

For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2007
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2008
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2009
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2010
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2011
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2012
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2013
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2014
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
2015
For depth 0 model level 0.0 will be used
For depth -100 model level -100.0 will be used
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  